__собираем предложения по квартирам__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/flat/avito' 
cian_raw_data_path = 'data/raw/flat/cian'

result_file_path = 'data/data_flat.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/flat/avito/avito_2022-08-26_17-29_raw.xlsx',
 'data/raw/flat/avito/avito_2022-08-29_13-16_raw.xlsx',
 'data/raw/flat/avito/avito_2022-08-31_17-42_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-01_14-07_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-02_13-04_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-05_14-02_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-06_13-09_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-07_12-39_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-08_13-36_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-12_13-44_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-13_13-55_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-15_10-27_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-16_15-07_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-19_12-36_raw.xlsx']

In [7]:
from lib.avito import AvitoDataCleanerRealtyFlat

data_avito = pd.concat([
        pd.read_excel(f)
        .dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

data_avito = AvitoDataCleanerRealtyFlat().transform( data_avito )

print(len(data_avito))

19375


In [8]:
assert len(data_avito)>0

In [9]:
data_avito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19375 entries, 0 to 19374
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   avito_id       19375 non-null  int64         
 1   title          19375 non-null  object        
 2   price          19375 non-null  int64         
 3   adr            19375 non-null  object        
 4   description    19375 non-null  object        
 5   obj_name       19375 non-null  object        
 6   ts             19375 non-null  datetime64[ns]
 7   avito_page     19375 non-null  int64         
 8   place          19375 non-null  object        
 9   nrooms         19375 non-null  int64         
 10  floor          19375 non-null  int64         
 11  nfloors        19375 non-null  int64         
 12  area           19375 non-null  float64       
 13  is_studio      19375 non-null  bool          
 14  is_apartment   19375 non-null  bool          
 15  is_part        1937

In [10]:
data_avito.isnull().any()

avito_id         False
title            False
price            False
adr              False
description      False
obj_name         False
ts               False
avito_page       False
place            False
nrooms           False
floor            False
nfloors          False
area             False
is_studio        False
is_apartment     False
is_part          False
is_auction       False
is_openspace     False
is_roof          False
is_SNT           False
priceM           False
is_last_floor    False
dtype: bool

In [11]:
# data_avito.query('nrooms==0')
# data_avito[ data_avito['title'].str.len()<1]

## собираем данные циан.ру

In [12]:
import re
from os import listdir

raw_data_files = sorted([
        cian_raw_data_path+'/'+f 
        for f in listdir(cian_raw_data_path) 
        if re.match(r'.+\.pkl$',f)
        #if re.match(r'.+\.xlsx$',f)
    
    ])
raw_data_files

['data/raw/flat/cian/cian_2022-09-08_14-25_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-12_13-59_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-13_13-59_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-15_10-30_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-16_15-11_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-19_12-40_raw.pkl']

In [13]:
data_cian = pd.concat([
        pd.read_pickle(f)
         .dropna()
         .drop_duplicates(['LinkArea','Description',])
         .rename(columns={'page':'cian_page'})
         .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

In [14]:
data_cian.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8995 entries, 0 to 8994
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   OfferTitle     8995 non-null   object        
 1   OfferSubtitle  8995 non-null   object        
 2   Deadline       8995 non-null   object        
 3   MainPrice      8995 non-null   object        
 4   PriceInfo      8995 non-null   object        
 5   GeoLabel       8995 non-null   object        
 6   TimeLabel      8995 non-null   object        
 7   LinkArea       8995 non-null   object        
 8   Description    8995 non-null   object        
 9   cian_page      8995 non-null   int64         
 10  ts             8995 non-null   datetime64[ns]
 11  place          8995 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 843.4+ KB


In [15]:
data_cian.isnull().any()

OfferTitle       False
OfferSubtitle    False
Deadline         False
MainPrice        False
PriceInfo        False
GeoLabel         False
TimeLabel        False
LinkArea         False
Description      False
cian_page        False
ts               False
place            False
dtype: bool

In [16]:
swap_title = data_cian['OfferSubtitle'].str.match(r'.*\d+ м², \d+/\d+ эт.*') 

data_cian = pd.concat([
        data_cian[ ~swap_title ],
        data_cian[ swap_title ]
            .rename(columns={'OfferTitle':'OfferSubtitle','OfferSubtitle':'OfferTitle'})
    ]).reset_index(drop=True)


In [17]:
# data_cian

In [18]:
from lib.cian import CianDataCleaner

data_cian = CianDataCleaner().transform( data_cian )

print(len(data_cian))

8995


In [19]:
assert len(data_cian)>0

In [20]:
data_cian.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8995 entries, 0 to 8994
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   title          8995 non-null   object        
 1   obj_name       8995 non-null   object        
 2   adr            8995 non-null   object        
 3   nrooms         8995 non-null   int64         
 4   floor          8995 non-null   int64         
 5   nfloors        8995 non-null   int64         
 6   area           8995 non-null   float64       
 7   is_studio      8995 non-null   bool          
 8   is_apartment   8995 non-null   bool          
 9   is_part        8995 non-null   bool          
 10  is_auction     8995 non-null   bool          
 11  is_openspace   8995 non-null   bool          
 12  is_roof        8995 non-null   bool          
 13  is_SNT         8995 non-null   bool          
 14  price          8995 non-null   int64         
 15  priceM         8995 n

In [21]:
data_cian.isnull().any()

title            False
obj_name         False
adr              False
nrooms           False
floor            False
nfloors          False
area             False
is_studio        False
is_apartment     False
is_part          False
is_auction       False
is_openspace     False
is_roof          False
is_SNT           False
price            False
priceM           False
is_last_floor    False
cian_url         False
description      False
cian_page        False
ts               False
place            False
dtype: bool

In [22]:
# data_cian
# data_cian.query('nrooms==0')
# data_cian[ data_cian['title'].str.len()<1 ]
# data_cian[ data_cian['obj_name'].str.len()>0 ][['title','obj_name',]] # .to_csv('tmp/cian_title.csv',sep='\t')|

----

In [23]:
data = pd.concat([data_avito,data_cian]).reset_index(drop=True) 
print(len(data))
assert len(data)>0

28370


## обновляем таблицу адресов

In [24]:
# data_ = data.sample(15).reset_index(drop=True)
# data_[['adr']]

In [25]:
places = { 
# поле 'place' и название города
'bahchisaray': 'Бахчисарай,',
      'sudak': 'Судак,',
  'sevastopol': 'Севастополь,',
}

def norm_place(d,places=places):
    if not d['place'] in places: return d['adr_']
    p = places[ d['place'] ]
    return d['adr_'] if d['adr_'].find(p)>-1 else p+d['adr_']

In [26]:
from lib.locator import AddressTransformer

atr = AddressTransformer()
data['adr_'] = data['adr'].progress_apply(atr.transform)
data['adr_'] = data[['adr_','place']].progress_apply(norm_place,axis=1)
data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

  0%|          | 0/28370 [00:00<?, ?it/s]

  0%|          | 0/28370 [00:00<?, ?it/s]

In [27]:
data[['adr_orig','adr']].sample(10)

,adr_orig,adr
21448,"Севастополь, район Ленинский, микрорайон Острякова, проспект Генерала Острякова, 66","Севастополь, район Ленинский, Острякова, проспект Генерала Острякова, 66"
7127,"Столетовский проспект , 29","Севастополь,Столетовский проспект , 29"
10990,"улица Горпищенко, 73","Севастополь, улица Горпищенко, 73"
17698,"шоссе Лабораторное, д. 33, секц. 4","Севастополь, шоссе Лабораторное, д. 33, секц. 4"
21307,"Севастополь, район Гагаринский, микрорайон Стрелецкая бухта, улица Степаняна, 4/2","Севастополь, район Гагаринский, Стрелецкая бухта, улица Степаняна, 4/2"
11372,"улица Горпищенко, д. 145, корп. 2","Севастополь, улица Горпищенко, д. 145, корп. 2"
22949,"Севастополь, район Ленинский, микрорайон Острякова, проспект Генерала Острякова, 172","Севастополь, район Ленинский, Острякова, проспект Генерала Острякова, 172"
21860,"Севастополь, район Ленинский, улица Генерала Крейзера","Севастополь, район Ленинский, улица Генерала Крейзера"
9666,"село Штурмовое, улица Комиссара Морозова, 10","Севастополь, село Штурмовое, улица Комиссара Морозова, 10"
21722,"Севастополь, район Гагаринский, улица Комбрига Потапова, 31 к3","Севастополь, район Гагаринский, улица Комбрига Потапова, 31 к3"


In [28]:
# from lib.locator import GeocoderSimpleOSM
from lib.locator import GeocoderYandex
from yandex_keys import keys

atr = AddressTransformer()
locator = GeocoderYandex(keys[0])
# locator = GeocoderSimpleOSM()

loc_file_path = 'data/location.pkl' # таблица адресов
loc = pd.read_pickle(loc_file_path)
# loc = pd.DataFrame([],columns=['adr','latitude','longitude',])
logging.info(f'LocationUpdater: {len(loc)} addresses in index')

# собираем все адреса в один список
loc = loc.merge(
        data[['adr']].drop_duplicates(),
        how='outer',
        on='adr',
    ).drop_duplicates().reset_index(drop=True)

logging.info(f'LocationUpdater: {len(loc)} addresses total')

# собираем все адреса с геопозицией
loc_def = loc[~loc['latitude'].isna()].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_def)} addresses defined')

# собираем все адреса без геопозиции
loc_undef = loc[loc['latitude'].isna()][['adr']].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_undef)} addresses undefined')

# определяем координаты по строке адреса
loc_undef = loc_undef.join( loc_undef['adr'].progress_apply(locator.transform).apply(pd.Series) ).dropna()     
logging.info(f'LocationUpdater: {len(loc_undef)} new addresses found')

loc = pd.concat([loc_def,loc_undef]).reset_index(drop=True) 

[INFO    ] 2022-09-20 00:40:23 | LocationUpdater: 1597 addresses in index
[INFO    ] 2022-09-20 00:40:23 | LocationUpdater: 3000 addresses total
[INFO    ] 2022-09-20 00:40:23 | LocationUpdater: 1597 addresses defined
[INFO    ] 2022-09-20 00:40:23 | LocationUpdater: 1403 addresses undefined


  0%|          | 0/1403 [00:00<?, ?it/s]

[INFO    ] 2022-09-20 00:47:12 | LocationUpdater: 1403 new addresses found


In [29]:
print( len(loc) )
loc.sample(3)

3000


,latitude,longitude,adr
2970,44.607408,33.526222,"Севастополь, район Ленинский, Центр, улица Терещенко, 1"
442,44.59,33.57,"Севастополь,Муромская улица , 80А"
2041,44.731955,33.548617,"Севастополь, село Орловка, Качинское шоссе , 35/49"


In [30]:
assert len(loc)>0
loc.to_pickle(loc_file_path)

## дополняем данные геометкой

In [31]:
# data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

In [32]:
data = data.merge(loc,on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isna() ] ) )

всего записей: 28370
записей без геометки: 0


In [33]:
assert len(data)>0
data.to_pickle(result_file_path)

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28370 entries, 0 to 28369
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   avito_id       19375 non-null  float64       
 1   title          28370 non-null  object        
 2   price          28370 non-null  int64         
 3   adr_orig       28370 non-null  object        
 4   description    28370 non-null  object        
 5   obj_name       28370 non-null  object        
 6   ts             28370 non-null  datetime64[ns]
 7   avito_page     19375 non-null  float64       
 8   place          28370 non-null  object        
 9   nrooms         28370 non-null  int64         
 10  floor          28370 non-null  int64         
 11  nfloors        28370 non-null  int64         
 12  area           28370 non-null  float64       
 13  is_studio      28370 non-null  bool          
 14  is_apartment   28370 non-null  bool          
 15  is_part        2837